# Part 1, Topic 3: Clock Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous lab, we learned how clock glitching can be used to get a microcontroller to skip a password check. In this lab, we'll see if we can glitch a more realistic target: a bootloader's command response.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

Now that we've got our feet wet with glitching, we're going to try something a bit more realistic: an "encrypted" bootloader (it's actually just rot-13, but we'll pretend it's unbreakable encryption), where we make as few assumptions as possible. Our goal will be to get that bootloader to decrypt the data and send it back to us. Here's what we know about the bootloader:

1. The `'p'` command is used to write encrypted firmware to the device. It takes in an encrypted ASCII-encoded string, terminated with a newline. Our first chunk of firmware is `"516261276720736265747267206762206f686c207a76797821"`.
1. It does *something* to it (presumably unencrypts it, authenticates it, etc. and writes it to memory)
1. It sends back an error code of `"r000000\n"`

Of immediate interest is that error code. That's the only time the bootloader communicates back with us, so attacking there is a good place to start. One thing that we'll assume is that we've got a trigger right before the error code is sent back to us. This is just a simple `trigger_high()` call, but we could also trigger on an IO line (better with the CW1200 Pro) or with a SAD trigger on a power trace (CW1200 Pro only). We've got a place to start, but let's see if we can learn more about the bootloader first.

We recommend using SimpleSerial V2 for this as, though the firmware doesn't use the simpleserial protocol, the faster baud rate will help speed up glitching.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER="SS_VER_2_1"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE -j SS_VER=$2

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../../../hardware/victims/firmware/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

The first thing we'll do is some simple power analysis to see what the device is doing when it sends data back to us. Serial communication is pretty slow, so set the ChipWhisperer to capture around 24k samples with a "x1" ADC clock.

In [ ]:
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

Next, capture a power trace. The string `"p516261276720736265747267206762206f686c207a76797821\n"` will send the bootloader the first chunk of code and plot it. If you don't see the full serial message, you can increase `scope.adc.decimate`, which will throw out every nth ADC sample.

In [ ]:
scope.adc.timeout = 3
scope.arm()
target.write("p516261276720736265747267206762206f686c207a76797821\n")
ret = scope.capture()
if ret:
    print("Timeout")
trace = scope.get_last_trace()


cw.plot(trace)

It doesn't look like anything too crazy is going on here - it's probably just printing some characters in a loop. Some ideas:

* If we glitch at the beginning of the loop, we might be able to corrupt the loop length variable and get it to print some extra memory
* We might be able to corrupt the loop variable and get it to read past where it's supposed to

For SimpleSerial V2, this should be short enough that you can quickly loop through the entirety of the code. If your target isn't using SimpleSerial V2, you should instead select a range a bit (~1000 cycles) before the end of the loop. If this doesn't succeed, you can try going after the cycles at the beginning of the loop.

**HINT: The last part of the loop should be near the beginning of the last power spike.**

**HINT: If you're really stuck on where the serial print ends, you can find the time between the `trigger_high()` and `trigger_low()` call with `scope.adc.trig_count`.**

In [ ]:
glitch_spots = [i for i in range(???)]
# ###################
# Add your code here
# ###################
raise NotImplementedError("Add your code here, and delete this.")

In [ ]:
print(glitch_spots)

### Evaluating Success

Detecting whether our glitch was successful or not isn't quite as trivial as in the previous lab - we don't have a nice error return that the device calculates and sends back to us. One idea is that we can look for part of the string that we sent to the device: there isn't much time between us sending it and the error code being returned. With any luck the compiler will have placed both values close in memory.

Now the rest is up to you! Use what you learned in the previous lab to setup glitch settings and a glitch loop. Here's a few hints to make things easier:

1. Try to use a fairly small width and offset range since we'll need to scan ext_offset as well here. A total range of ~2-3 for each with 0.4 steps is a good range to aim for. These numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1.
1. Try looking for a part of the string we sent to the device to check for success.
1. You may want to forgo graphing or plot only successes/crashes if it makes things substantially slower - we're scanning a large range of glitch settings so we'll need all the speed we can get.

Set your glitch up here:

In [ ]:
scope.adc.timeout = 0.1

scope.cglitch_setup()

def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')

Again, we can use the glitch controller to make loop setup easier:

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

In [ ]:
x_bound = (-48, 48)
y_bound = (glitch_spots[0], glitch_spots[-1])
if scope._is_husky:
    x_bound = gc.set_range("width", 3900, 4500)
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_bound=x_bound, y_bound=y_bound,
               x_index="width", y_index="ext_offset")

In [ ]:
# replace with good glitch settings you've found
gc.set_range("width", ???, ???)
gc.set_range("offset", ???, ???)
gc.set_global_step(???)

gc.set_range("ext_offset", glitch_spots[0], glitch_spots[-1])
gc.set_step("ext_offset", glitch_spots[1] - glitch_spots[0])
gc.set_range("tries", 1, 1)
gc.set_step("tries", 1)
gc.set_step("ext_offset", 1)
scope.glitch.repeat = 1

    
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    if broken:
        break
    if scope.adc.state:
        #print("Timeout, trigger still high!")
        gc.add("reset")
        #Device is slow to boot?
        reboot_flush()
        
    target.flush()
    scope.arm()
    target.write("p516261276720736265747267206762206f686c207a76797821\n")
    ret = scope.capture()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        time.sleep(0.05)
        output = target.read(timeout=2)
        if ??? in output:
            print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            gc.add("success")
            broken = True 
            for __ in range(500):
                num_char = target.in_waiting()
                if num_char:
                    my_print(output)
                    output = target.read(timeout=50)
            time.sleep(1)
            break
        else:
            gc.add("normal")

## Diagnosing the Fault

As you can see by the output, the bootloader has suffered a pretty catastrophic failure! Not only has it spilled the secret, it's also dumped a whole bunch more memory. For a real bootloader, there's probably some pretty juicy stuff in there like encryption keys or previously decrypted firmware. Let's start by taking a look at the C source code that sends the error code back:

```C
trigger_high();

int i;
for(i = 0; i < ascii_idx; i++)
{
    putch(ascii_buffer[i]);
}
trigger_low();
state = IDLE;
```

Nothing really looks too unusual here. Before we take a look at the assembly and figure out what went wrong, let's try to make some guesses:

* Maybe the glitch corrupted the `ascii_idx` variable
    * The glitch happened near the end of the loop. It's unlikely the end of loop counter would be reloaded during the loop
* Maybe we skipped the last `i < ascii_idx` check
    * The glitch caused **a lot** of memory to be dumped. If we just skipped the last check it **should** only print an extra character
* i is a signed integer: maybe we corrupted it into being a really large negative number.

That last one seems to be our best theory, so let's go with that.

## The Answer

Let's check the assembly for our booloader. No need to decompile the binary or recompile to assembly, since there's also a listing file created as part of the build process (`*.lss`). This file also contains C, so it makes it easy to search (try something like the `trigger_high()` call). You might notice that instead of doing a `less than or equal` or `less than` comparison like was in our C code, the compiler has instead inserted a `not equal` comparison instead! This means our original guess may not have been correct, as our assumption about what would happen if the last `i < ascii_idx` was skipped doesn't hold. In fact, it's a lot more likely that the last check was skipped (or i was set to some large value) than flipping a particular bit.

This is actually a pretty unexpected change for the compiler to make, espcially since `less than`, `greater than`, and `not equal` are nearly identical instructions in terms of implementation and have both the same instruction size and speed. This showcases an important fact: the C code that you write is not directly translated to assembly. It needs to go through the compiler first, which may drastically change the intended logic of the program.

Now that we know what happened, let's look at some ways to fix it.

### 1. Volatile variables

C includes a keyword for variables called `volatile`, which indicates that the variable may change between accesses and therefore should not have optimizations applied to it. A typical use case for `volatile` is for peripheral registers on embedded devices. It would be really bad, for example, if you were trying to wait for an IO pin to go high in your code, but the compiler decided it would be faster to only check it only once and assume it doesn't change!

Try replacing `int i = 0;` before the print look with `volatile int i = 0;`, recompile, and check the listing file. Is there any other unexpected changes? What about if you consider the use case above (i.e. if `i` was a register instead of a loop variable)? Is there any way the attack might still work? If so, how might you mitigate this?

### 2. Unrolling the loop

Another potential way of solving this issue would be to manually unroll the loop. The message being printed by the bootloader is a constant length of 7 characters, so we could instead write:

```C
int i;
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
```

In fact, this is something the compiler might do on its own to optimize the code, since unrolling a loop like this is faster than the loop version. It's not a good idea to blindly rely on this, however, since the compiler could choose not to make this optimization as well and might change it between builds.

### 3. Checking for invalid characters

Another thing to consider is that the message from the bootloader only has a limited range of characters that it prints. We could instead construct a "safe print" function that only prints newlines, `'r'` and ASCII digits (i.e. `'0'` to `'9'`):

```C
int safe_print(char c)
{
    if ((c == '\n') ||
       ((c >= '0') && (c <= '9')) ||
       (c == 'r')) {
        putch(c);
        return 0;
    }
    return -1; //uh oh!
}
```

It we went this route, it would be a good idea to make the error return a separate buffer with a bunch of null characters at the end.

### 4. More generic methods

More generic ways of defending against glitch attacks (memory guards, for example) are also discussed in the training slides.

In [ ]:
scope.dis()
target.dis()